In [62]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

# templete = PromptTemplate.from_template("{country_a}과 {country_b}의 거리는 얼마나되니? 그리고 네 이름은 무엇이니?")
# prompt = templete.format(country_a="멕시코", country_b="태국")

# chat.predict(prompt)

# # import some message constructors
# from langchain.schema import HumanMessage, AIMessage, SystemMessage

# messages = [
#   SystemMessage(content="You are a geography expert. And you only reply in Korean"),
#   AIMessage(content="Hi, my nick name is Liberty!"),
#   HumanMessage(content="남한과 북한의 거리는 얼마나되니? 그리고 네 이름은 무엇이니?")
# ]

# chat.predict_messages(messages)

templete = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}"),
        ("ai", "Hi, my nick name is {name}!"),
        (
            "human",
            "{country_a}과 {country_b}의 거리는 얼마나되니? 그리고 네 이름은 무엇이니?",
        ),
    ]
)

prompt = templete.format_messages(
    language="Korean", name="Socrates", country_a="Mexico", country_b="Thailand"
)

chat.predict_messages(prompt)

Mexico와 Thailand 사이의 거리는 대략 16,000km 정도입니다. 제 이름은 소크라테스입니다. 어떤 도움이 필요하신가요?

AIMessageChunk(content='Mexico와 Thailand 사이의 거리는 대략 16,000km 정도입니다. 제 이름은 소크라테스입니다. 어떤 도움이 필요하신가요?')

In [63]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):

  def parse(self, text):
    items = text.strip().split(",")
    return list(map(str.strip, items))




template = ChatPromptTemplate.from_messages([
  ("system", "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items} in lowercase. Do NOT reply with anything else."),
  ("human", "{question}")
])

# prompt = template.format_messages(max_items=10, question="What are the colors?")

# result = chat.predict_messages(prompt)

# p = CommaOutputParser()

# p.parse(result.content)

chain = template | chat | CommaOutputParser()
chain.invoke({"max_items":5, "question":"What are the poketmons?"})


pikachu, charmander, bulbasaur, squirtle, jigglypuff

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']

In [64]:
investor_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "너는 세계적인 주식투자자야. 특히 한국 주식시장에서 스켈핑, 단타, 스윙에 아주 능숙해. 그리고 넌 뉴스를 읽고서 주가가 상승할 재료를 찾고 당일의 테마를 확인하는데 특출났어!",
        ),
        ("human", "과거와 오늘 뉴스들을 통해서 {stock_date} 주가 상승이 예상되는 종목과 테마를 찾고싶어.")
    ]
)

investor_chain = investor_prompt | chat




In [65]:
chart_investor_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "너는 세계적인 투자자이며 기술적 분석에 특화되어있어. 특정 종목이 급등할 때의 거래량과 차트가 어떤 현상이 나타나며 과거에 비슷한 history가 있었는지를 알고싶어. 그리고 평소와 어떻게 다른지도. 잘 모르겠다면 모르겠다고 해줘.",
        ),
        ("human", "{recipe}"),
    ]
)

chart_investor_chain = chart_investor_prompt | chat

final_chain = {"recipe": investor_chain} | chart_investor_chain
final_chain.invoke({"stock_date": "내일"})



알겠어, 몇 가지 뉴스를 살펴보고 내일 주가 상승이 예상되는 종목과 테마를 찾아보자.

1. 삼성전자: 최근 삼성전자가 반도체 시장에서의 경쟁력을 강화하기 위해 투자를 확대하고 있는 소식이 있었어. 또한, 최근 발표된 실적이 예상을 상회했고, 미래 성장 가능성에 대한 기대도 높아지고 있어. 내일 삼성전자 주가 상승이 예상될 수 있을 것 같아.

2. 바이오의약품 기업: 최근 코로나19 백신 관련 뉴스가 계속해서 나오고 있어. 특히 한국의 바이오의약품 기업들이 백신 개발에 성공적인 결과를 얻고 있는 것으로 알려져 있어. 내일 바이오의약품 관련 기업들의 주가 상승이 예상될 수 있을 것 같아.

3. 5G 관련 기업: 최근 5G 기술이 더욱 확산되고 있는 가운데, 관련 기업들의 주가도 상승세를 보이고 있어. 내일 5G 관련 기업들의 주가 상승이 예상될 수 있을 것 같아.

4. 친환경 에너지 기업: 환경 문제에 대한 관심이 높아지면서 친환경 에너지 기업들의 주가도 상승세를 보이고 있어. 내일 친환경 에너지 기업들의 주가 상승이 예상될 수 있을 것 같아.

이러한 종목들을 중심으로 내일의 시장을 주의 깊게 살펴보면 좋을 것 같아. 당일의 뉴스와 시장 상황에 따라서 투자 결정을 내리는 것이 중요하니까 신중하게 판단해야 해.주어진 정보를 기반으로 내일 주가 상승이 예상되는 종목과 테마를 찾아보겠어.

1. 삼성전자: 삼성전자의 경우 반도체 시장에서의 경쟁력 강화와 실적 상회 소식으로 내일 주가 상승이 예상될 수 있습니다. 기술적 분석에서는 거래량이 증가하고 차트에서 상승 신호가 나타날 경우 급등할 가능성이 있습니다. 이러한 상황이 과거에도 있었는지 확인해보면 좋을 것 같아.

2. 바이오의약품 기업: 코로나19 백신 관련 뉴스와 성공적인 결과 소식으로 내일 바이오의약품 기업들의 주가 상승이 예상될 수 있습니다. 이 역시 기술적 분석에서는 거래량과 차트를 확인하여 급등 가능성을 파악할 수 있습니다.

3. 5G 관련 기업: 5G 기술의 확산과 주가 상승세를 고려할 때 내일

AIMessageChunk(content='주어진 정보를 기반으로 내일 주가 상승이 예상되는 종목과 테마를 찾아보겠어.\n\n1. 삼성전자: 삼성전자의 경우 반도체 시장에서의 경쟁력 강화와 실적 상회 소식으로 내일 주가 상승이 예상될 수 있습니다. 기술적 분석에서는 거래량이 증가하고 차트에서 상승 신호가 나타날 경우 급등할 가능성이 있습니다. 이러한 상황이 과거에도 있었는지 확인해보면 좋을 것 같아.\n\n2. 바이오의약품 기업: 코로나19 백신 관련 뉴스와 성공적인 결과 소식으로 내일 바이오의약품 기업들의 주가 상승이 예상될 수 있습니다. 이 역시 기술적 분석에서는 거래량과 차트를 확인하여 급등 가능성을 파악할 수 있습니다.\n\n3. 5G 관련 기업: 5G 기술의 확산과 주가 상승세를 고려할 때 내일 5G 관련 기업들의 주가 상승이 예상될 수 있습니다. 기술적 분석에서도 이러한 추세를 확인할 수 있을 것입니다.\n\n4. 친환경 에너지 기업: 환경 문제에 대한 관심 상승으로 친환경 에너지 기업들의 주가도 상승세를 보이고 있습니다. 내일 주가 상승이 예상될 수 있으며, 기술적 분석을 통해 추가적인 신호를 확인할 수 있을 것입니다.\n\n이러한 종목들을 중심으로 내일의 시장을 살펴보면 좋을 것 같아. 추가적인 기술적 분석을 통해 상승 가능성을 더욱 정확히 예측할 수 있을 것이니 참고하시기 바랍니다.')